# Spanner Workbench Presentation

### Presenting:
| #       |              Name |             Id |             Email |
|---------|-------------------|----------------|------------------ |
|Student 1|  Tom Feldman | 325135960 | tom.p@campus.technion.ac.il |
|Student 2|  Niv Shetrit | 316354976 | shniv@campus.technion.ac.il |

TODO@niv
## Presentation Overview:
* RGXlog's purpose
* How RGXlog Looked Like Before Our Changes
* Information Extraction Functions
* Import-Export Functions
* Standard Library
* Tests, Typing, and Documentation
* **PyDatalog to SQL Engine**:
    * **Parse Grpah and Term Graph**
    * **Adding Rules to Trem Graph**
    * **SQL Engine**
    * **Bottom-Up Execution**
* New Engine Features
* Tutorials Overview

# google doc notes: 
https://docs.google.com/document/d/1zH5byp0EQYoAq2ihWS9AXWQEtcroiZ7oy1z41whT4Sw/edit?usp=sharing

## Presentation Overview:
* **RGXlog's purpose**
* How RGXlog Looked Like Before Our Changes
* Information Extraction Functions
* Import-Export Functions
* Standard Library
* Tests, Typing, and Documentation
* PyDatalog to SQL Engine:
* New Engine Features
* Tutorials Overview

# RGXlog's Purpose

* Incorporating information extraction in the relational database model
* Our language will be a framework used by other programmers to implement their own version of Spannerlog
* Our implementation is very modular in order to allow others to modify it however they like

# How RGXlog Looked Like Before Our Changes
* the bare-minimum working version of the project
* relied heavily on pydatalog, which isn't scalable
* adding IE functions was not supported - required modifying the code itself
* recursive rules were not supported
* no tests at all



# TODO: for every feature, explain why it was needed

# Information Extraction Functions <a class="anchor" id="IEFunctions"></a>

* IE-functions are the very essence of RGXLog. It is necessary to let users add them easily and programmatically 
* Client-Server refactoring
* Design of `IEFunction` Class

TODO@tom: (in contrast to manually editing the code)

## Example: Logical Operators:

#### Implemention of the **not equals (NEQ)** relation.

We would like to have a rule such as:
<br>
```unique_pair(X, Y) <- pair(X, Y), X != Y```
<br><br>
Unfortunately RGXLog doesn't support True/False values. Therefore, we can't use ```X != Y```.
<br>
Our solution to this problem is to create an ie function that implements NEQ relation:

In [15]:
import rgxlog
from rgxlog.engine.datatypes.primitive_types import DataTypes

def NEQ(x: str, y: str):
    if x == y:
        # return false (empty tuple represents false)
        yield tuple() 
    else:
        #return true
        yield x, y

in_out_types = [DataTypes.string, DataTypes.string]
rgxlog.magic_session.register(ie_function=NEQ, 
                       ie_function_name="NEQ", 
                       in_rel=in_out_types, 
                       out_rel=in_out_types)

In [2]:
%%rgxlog
new pair(str, str)
pair("Dan", "Tom")
pair("Cat", "Dog")
pair("Apple", "Apple")
pair("Cow", "Cow")
pair("123", "321")

unique_pair(X, Y) <- pair(First, Second), \
                     NEQ(First, Second) -> (X, Y)
?unique_pair(X, Y)

printing results for query 'unique_pair(X, Y)':
  X  |  Y
-----+-----
 Dan | Tom
 Cat | Dog
 123 | 321



# Import-Export Functions <a class="anchor" id="Import-Export-Functions"></a>

* RGXlog is used as a data extraction/manipulation language, in which people use spreadsheets and dataframes very often
* import/export a relation from a csv file
* import/export a relation from a dataframe

In [3]:
%%bash
cat enrolled.csv

abigail,operating_systems
jordan,chemistry
gale,operating_systems
howard,chemistry
howard,physics


In [4]:
rgxlog.magic_session.import_relation_from_csv("enrolled.csv", relation_name="enrolled", delimiter=",")

In [5]:
%%rgxlog
enrolled("abigail", "chemistry")
gpa_str = "abigail 100 jordan 80 gale 79 howard 60"

gpa(Student, Grade) <- py_rgx_string(gpa_str, "(\w+).*?(\d+)")->(Student, Grade), enrolled(Student, X)

?gpa(X, Y)

printing results for query 'gpa(X, Y)':
    X    |   Y
---------+-----
 abigail | 100
 jordan  |  80
  gale   |  79
 howard  |  60



# Standard Library <a class="anchor" id="STDLIB"></a>

* contains implementation of many default IE functions, which users can use as a template to create their own functions
* python and rust regex
* json path
* many nlp functions wrapping StanfordCoreNLP


TODO@tom: rust regex shows users how to create IE functions which use unix functions. json/stanford show how to wrap pythonic functions

## NLP Example - Named Entity Recognition:

NER IEFunction Recognizes named entities (person and company names, etc.)

In [6]:
from rgxlog.stdlib.nlp import CoreNLPEngine

def ner_wrapper(sentence: str):
    for res in CoreNLPEngine.ner(sentence):
            yield res["token"], res["ner"], res["span"] 

In [7]:
rgxlog.magic_session.register(ie_function=ner_wrapper,
                       ie_function_name='NER',
                       in_rel=[DataTypes.string],
                       out_rel=[DataTypes.string, DataTypes.string, DataTypes.span])

In [8]:
%%rgxlog

sentence = "While in France, Christine Lagarde discussed short-term stimulus  \
            efforts in a recent interview with the Wall Street Journal."
                
ner(Token, NER, Span) <- NER(sentence)->(Token, NER, Span)
?ner(Token, NER, Span)

printing results for query 'ner(Token, NER, Span)':
   Token   |     NER      |    Span
-----------+--------------+------------
   While   |      O       |   [0, 5)
    in     |      O       |   [6, 8)
  France   |   COUNTRY    |  [9, 15)
     ,     |      O       |  [15, 16)
 Christine |    PERSON    |  [17, 26)
  Lagarde  |    PERSON    |  [27, 34)
 discussed |      O       |  [35, 44)
   short   |      O       |  [45, 50)
     -     |      O       |  [50, 51)
   term    |      O       |  [51, 55)
 stimulus  |      O       |  [56, 64)
  efforts  |      O       |  [78, 85)
    in     |      O       |  [86, 88)
     a     |      O       |  [89, 90)
  recent   |      O       |  [91, 97)
 interview |      O       | [98, 107)
   with    |      O       | [108, 112)
    the    |      O       | [113, 116)
   Wall    | ORGANIZATION | [117, 121)
  Street   | ORGANIZATION | [122, 128)
  Journal  | ORGANIZATION | [129, 136)
     .     |      O       | [136, 137)



# Tests, Typing, and Documentation <a class="anchor" id="CI"></a>

* our codebase has grown significantly over the last iteration, and we want to make sure that it is consistently stable and easy to understand
* this is why we are running github actions CI after every push, which include:
    * 63 unique pytest tests
    * pep8 test
    * mypy test - all functions are type annotated
    
* also, all of our functions are documented according to 
 <a href="http://epydoc.sourceforge.net/manual-epytext.html">the epytext standard</a>, which makes it easy for us to <a href="https://pypi.org/project/epydoc/">export it to html</a>

* a successful run looks like this:

<img src="git_workflow.png" alt="workflow" width="600" height="600" style="display: block; margin-left: auto; margin-right: auto; width: 50%;">

TODO@niv
## Presentation Overview:
* RGXlog's purpose
* How RGXlog Looked Like Before Our Changes
* Information Extraction Functions
* Import-Export Functions
* Standard Library
* Tests, Typing, and Documentation
* **PyDatalog to SQL Engine**:
    * **Parse Grpah and Term Graph**
    * **Adding Rules to Trem Graph**
    * **SQL Engine**
    * **Bottom-Up Execution**
* New Engine Features
* Tutorials Overview

# PyDatalog to SQL Engine <a class="anchor" id="Engine"></a>

* previously, we were heavily dependent on pydatalog, which doesn't enable access to its internal calculations

which is why we've implemented our main feature, which contains:
* parse graph and term graph
* SQL engine
* adding rules to term graph
* bottom-up algorithm and execution function

## parse and term graph

we created the data structures which store information for our calculations:
* parse graph - stores facts, relation declerations, rule declarations and queries.
* term graph - stores connection between all the relations.

In [9]:
from rgxlog import Session

rgxlog.magic_session = Session()  # reset session

In [10]:
%%rgxlog 

new parent(str, str)
parent("Liam", "Noah")
parent("Noah", "Oliver")
parent("Oliver", "Mason")

ancestor(X,Y) <- parent(X,Y)
ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)
?ancestor(Parent, Son)

printing results for query 'ancestor(Parent, Son)':
  Parent  |  Son
----------+--------
   Liam   | Mason
   Liam   |  Noah
   Liam   | Oliver
   Noah   | Mason
   Noah   | Oliver
  Oliver  | Mason



In [11]:
print(f"parse graph:\n{rgxlog.magic_session._parse_graph}")

parse graph:
(__rgxlog_root) (computed) root
    (0) (computed) relation_declaration: parent(str, str)
    (1) (computed) add_fact: parent("Liam", "Noah")
    (2) (computed) add_fact: parent("Noah", "Oliver")
    (3) (computed) add_fact: parent("Oliver", "Mason")
    (4) (computed) rule: ancestor(X, Y) <- parent(X, Y)
    (5) (computed) rule: ancestor(X, Y) <- parent(X, Z), ancestor(Z, Y)
    (6) (computed) query: ancestor(Parent, Son)



In [12]:
print(f"term graph:\n{rgxlog.magic_session._term_graph}")

term graph:
(__rgxlog_root) (not_computed) root
    (ancestor) (not_computed) rule_rel: ancestor(X, Y)
        (0) (not_computed) union
            (1) (not_computed) project: ['X', 'Y']
                (2) (not_computed) get_rel: parent(X, Y)
            (3) (not_computed) project: ['X', 'Y']
                (4) (not_computed) join: {'X': [(parent(X, Z), 0)], 'Z': [(parent(X, Z), 1), (ancestor(Z, Y), 0)], 'Y': [(ancestor(Z, Y), 1)]}
                    (5) (not_computed) get_rel: parent(X, Z)
                    (6) (not_computed) get_rel: ancestor(Z, Y)
                        (ancestor) (not_computed) rule_rel: ancestor(X, Y)

DependencyGraph is:
__rgxlog_root
    ancestor



## Adding rules to Term Graph

* traverse the parse graph and find new rules
* compute a legal execution order of a rule (```A(X, Y) <- B(Z) -> (X), C(Y), D(Y, Z)```)
* join all the body relations in the computed order
* project to the relevant variables
* connect to the rule head node
* select from a relation if it contains constant term (```A(x) <- B(X,1)```)
* calculate ie relations using its bounding relations

In [13]:
print(f"term graph:\n{rgxlog.magic_session._term_graph}")

term graph:
(__rgxlog_root) (not_computed) root
    (ancestor) (not_computed) rule_rel: ancestor(X, Y)
        (0) (not_computed) union
            (1) (not_computed) project: ['X', 'Y']
                (2) (not_computed) get_rel: parent(X, Y)
            (3) (not_computed) project: ['X', 'Y']
                (4) (not_computed) join: {'X': [(parent(X, Z), 0)], 'Z': [(parent(X, Z), 1), (ancestor(Z, Y), 0)], 'Y': [(ancestor(Z, Y), 1)]}
                    (5) (not_computed) get_rel: parent(X, Z)
                    (6) (not_computed) get_rel: ancestor(Z, Y)
                        (ancestor) (not_computed) rule_rel: ancestor(X, Y)

DependencyGraph is:
__rgxlog_root
    ancestor



### Execution Order of Rule Example:
 
* ```A(X, Y) <- B(Z) -> (X), C(Y), D(Y, Z)``` isn't safe since the term **Z** in ie relation **B** isn't bounded.
* ```A(X, Y) <- C(Y), D(Y, Z), B(Z) -> (X)``` is safe since the term **Z** is bounded by relations **C and D**.

* A legal execution order is computed using iterative algorithm that finds boinding relations to all of the ie relations.

## SQL engine
* as mentioned before, the old pydatalog implementation was lacking due to a limited interface
* in our new implementation, we have direct access to the data, which is stored in SQL tables
* each relation is stored in a separate SQL table
* rules' outputs are calculated using a chain of SQL operators
* thanks to the `Jinja` library, we can use neat SQL templates in our code

### SQL engine method implementation example: add_fact

```python
def add_fact(self, fact: AddFact) -> None:
        num_types = len(fact.type_list)
        col_names = [f"{self._get_col_name(i)}" for i in range(num_types)]
        col_values = [self._convert_relation_term_to_string(datatype, term) 
                      for datatype, term in zip(fact.type_list, fact.term_list)]

        template_dict = {"col_values": col_values,
                         "fact": fact, "col_names": col_names}
```

```python
sql_template = ("""
        INSERT INTO {{fact.relation_name}} ({{col_names | join(", ")}})
        VALUES ({{col_values | join(", ")}})
        """)

        self._run_sql_from_jinja_template(sql_template, template_dict)
```

## Execution Function

* Implements a naive bottom-up alogrithm:
    * reset all the mutually recursive relations
    * traverse the term graph and update all the mutually recursive relations based on the previous step
    * stop when all the mutually recursive relations converged at the same step.
* uses the SQL engine

## Example - Ancestor Program

$ancestor_{i}(X,Y)$ <- $parent(X,Y)$ <br>
$ancestor_{i}(X,Y)$ <- $parent(X,Z), ancestor_{i - 1}(Z,Y)$

### ancestor_0 (empty table):

| Parent | Son |
| --- | --- |
|     |     |

### ancestor_1 (finds parents):


| Parent | Son |
| --- | --- |
| Liam | Noah |
| Noah | Oliver |
| Oliver | Mason |

### ancestor_2 (finds grandparents):

| Parent | Son |
| --- | --- |
| Liam | Noah |
| Noah | Oliver |
| Oliver | Mason |
| Liam | Oliver |
| Noah | Mason |

### ancestor_3 (finds great-grandparents):

| Parent | Son |
| --- | --- |
| Liam | Noah |
| Noah | Oliver |
| Oliver | Mason |
| Liam | Oliver |
| Noah | Mason |
| Liam | Mason |

### ancestor_4 (finds great-great-grandparents):

| Parent | Son |
| --- | --- |
| Liam | Noah |
| Noah | Oliver |
| Oliver | Mason |
| Liam | Oliver |
| Noah | Mason |
| Liam | Mason |

## The computation is stopped since no tuples were added (fixed point reached)

## New Engine Features
we've also added tools which are useful for both users and developers (more examples in the tutorials):
* remove rules - `session.remove_rule`
* loggers/debug info
* enable union of rules (same head)
```
ancestor(X,Y) <- parent(X,Y)
ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)
```
* print rules and ie functions - `session.print_all_rules`, `session.print_registered_ie_functions`

## Tutorials Overview

<a href="./introduction.ipynb">introduction</a><br>
<a href="./Advanced usage.ipynb">advanced</a>